#1. Prepare Python that loads naics4 data into Pandas for 2017 to 2021 for Maine
Source files. Load these directly from the URL into Pandas.

https://model.earth/community-data/industries/naics/US/counties/ME/US-ME-census-naics4-counties-2021.csv

View files on GitHub: industries/naics/US/counties/ME

Add your first name instead of "TO DO" if you are working on one of these:  

Lily: Rename Sqkm to "Km2". That works well for these descriptions. Thanks!

Lily: Capitalize the first letter of all the columns and remove underscore by adding a second CapitalLetter. (Underscores are not compatible with README.md syntax, they require a \ before them.)

Lily: Round all the numbers to 2-decimal places, including the Latitude and Longitude.

Lily: Divide "Population" by 1000 so the data is smaller.

Sijia: For the following, the cenpy library is suggested:  
https://chat.openai.com/share/289c1005-510d-44bb-a6f9-4cca1c0b6e5f

For "UrbanPercent" we'd need to calculate the percentage of the land area in a county is urban (with over 1500 people). We can get this using census blocks which are subsections of counties.  Get the size of each census block in the county with a population of 1500 and over. Add those to get the UrbanKm2.  Then divide by Km2 to get the UrbanPercent:

UrbanPercent = UrbanKm2 / Km2

The maximum "UrbanPercent" would be 1 since it is the ratio for a county that is 100% urban, like NYC.

Sijia: Fix shape file read-in path in get_area. Do not need to download shapefile for input now, instead read from census.gov url.

TO DO: Generate for both naics2 and naics4. Deploy to a fork of community-timelines for review.

In [ ]:
import pandas as pd
import geopandas as gpd
import requests
from io import StringIO
from geopy.geocoders import Nominatim
import os, re
import numpy as np

In [ ]:
def get_df(url):
    try:
        response = requests.get(url)
        df = pd.read_csv(StringIO(response.text))
        return df
    except:
        return np.nan

# add cencus: name, pop, long, lat, area

In [ ]:
!pip install cenpy

In [ ]:
def get_name_population_cencusdata(year,fips):
    try:
        import censusdata
        state_fips_code = str(fips)[0:-3]
        county_fips_code= str(fips)[-3:]
        data = pd.DataFrame(censusdata.download('acs5', year, censusdata.censusgeo([('state', state_fips_code), ('county',county_fips_code )]), ['B01003_001E']))
        data=data.reset_index().rename(columns={'index': 'county',"B01003_001E":"total_population"})
        data.county=data.county.apply(lambda x: str(x).split(",")[0])
        return data.county[0],data.total_population[0]
    except:
            return '',np.nan

In [ ]:
def get_name_population_cenpy(year,fips):
    try:
        import cenpy
        state_code=str(fips)[0:-3]
        county_code=str(fips)[-3:]
        # Set the connection to the Census API for the specific year
        conn = cenpy.products.APIConnection(f"ACSDT5Y{year}")
        # Get the county population data for all counties in the US for the current year
        df = conn.query(cols=["NAME", "B01001_001E"], geo_filter={"state":state_code},geo_unit=f"county:{county_code}")
        name=df.NAME[0].split(",")[0]
        pop=int(df.B01001_001E[0])
        return name,round(pop/1000,2)
    except:
        return "",np.nan

In [ ]:
def get_long_lat(name):
    try:
        geolocator = Nominatim(user_agent="my_geocoder")
        county_coordinates = {}
        location = geolocator.geocode(name)
        lat=round(location.latitude,2)
        lon=round(location.longitude,2)
        return lon,lat
    except:
        return np.nan,np.nan

In [ ]:
def get_area(year, state_fp):
    test = gpd.read_file(f'https://www2.census.gov/geo/tiger/GENZ{year}/shp/cb_{year}_us_county_500k.zip')
    tost = test.copy()
    tost= tost.to_crs({'init': 'epsg:3035'})
    tost["Km2"] = round(tost['geometry'].area/ 10**6,2)
    tost=tost[tost.STATEFP==state_fp]
    tost["Fips"]=tost.STATEFP+tost.COUNTYFP
    return tost[["Fips","Km2"]]

In [ ]:
def get_block_group_area(year, state_fp):
    block_groups = gpd.read_file(f'https://www2.census.gov/geo/tiger/GENZ{year}/shp/cb_{year}_{state_fp}_bg_500k.zip')
    block_groups_copy = block_groups.copy()
    block_groups_copy = block_groups_copy.to_crs({'init': 'epsg:3035'})
    block_groups_copy["Km2"] = round(block_groups_copy['geometry'].area/ 10**6,2)
    block_groups_copy["Fips"]= block_groups_copy.STATEFP+block_groups_copy.COUNTYFP
    return block_groups_copy[["Fips","GEOID", "Km2"]]

In [ ]:
def get_full_code(input_string):
    pattern = r'state:(\d+)> county:(\d+)> tract:(\d+)> block group:(\d+)'
    # Use re.search to find the match
    match = re.search(pattern, input_string)
    # Extract the codes
    state_code = match.group(1)
    county_code = match.group(2)
    tract_code = match.group(3)
    block_group_code = match.group(4)
    # Concatenate the codes
    full_code = state_code + county_code + tract_code + block_group_code
    return full_code

In [ ]:
def get_census_block_population(year,state_fp):
    import censusdata
    data = pd.DataFrame(censusdata.download('acs5', year, censusdata.censusgeo([('state', state_fp), ('county','*'), ('block group', '*')]), ['B01003_001E']))
    data=data.reset_index().rename(columns={'index': 'GEOID',"B01003_001E":"total_population"})
    data.GEOID=data.GEOID.apply(lambda x: get_full_code(str(x)))
    return data

In [ ]:
def get_urban_percent(year,state_fp):
    block_gp_area = get_block_group_area(year, state_fp)
    block_gp_population = get_census_block_population(year,state_fp)
    df = block_gp_area.merge(block_gp_population, on="GEOID", how="inner")
    df_result = pd.DataFrame(round(df[df["total_population"] >= 1500].groupby("Fips")["Km2"].sum()/df.groupby("Fips")["Km2"].sum(),2))
    df_result = df_result.reset_index().rename(columns={'index': 'Fips',"Km2":"UrbanPercent"})
    return df_result

# main funct

In [ ]:
def main(year,state,naics_value):
    url = f"https://model.earth/community-data/industries/naics/US/counties/{state}/US-{state}-census-naics{naics_value}-counties-{year}.csv"
    df=get_df(url)
    df=df.pivot_table(index="Fips", columns='Naics', values=["Establishments","Employees","Payroll"]).reset_index()
    df.columns = [f"{a}-{str(b)}" for a,b in df.columns]
    sorted_columns = ["Fips-"]+sorted(df.columns[1:], key=lambda x: int(x.split('-')[-1]))
    df = df[sorted_columns].rename(columns={"Fips-":"Fips"})
    df.insert(loc=1, column='Name', value=df.Fips.apply(lambda x:get_name_population_cencusdata(year,x)[0]))
    df.insert(loc=2, column='Population', value=df.Fips.apply(lambda x:get_name_population_cencusdata(year,x)[1]))
    df.insert(loc=3, column='Longitude', value=df.Name.apply(lambda x:get_long_lat(x)[0]))
    df.insert(loc=4, column='Latitude', value=df.Name.apply(lambda x:get_long_lat(x)[1]))
    state_code=str(df.Fips[0])[0:2]
    area_df=get_area(year,state_code)
    df.Fips=df.Fips.astype(str)
    df=df.merge(area_df,how="left",on="Fips")
    column_data = df.pop('Km2')
    df.insert(loc=5, column='Km2', value=column_data)
    df.insert(loc=6, column='UrbanDensity', value=df.apply(lambda x:round(x.Population/x.Km2,2),axis=1))
    urban_df=get_urban_percent(year,state_code)
    df=df.merge(urban_df,how="left",on="Fips")
    column_data = df.pop('UrbanPercent')
    df.insert(loc=7, column='UrbanPercent', value=column_data)
    display(df)
    """
    output_dir = f"../output/{year}"
    os.makedirs(output_dir, exist_ok=True)
    path = f"{output_dir}/US-{state}-training-naics{naics_value}-counties-{year}.csv"
    df.to_csv(path, header=True, index=False)
    """

In [ ]:
main(2017,"ME",4)

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Fips,Name,Population,Longitude,Latitude,Km2,UrbanDensity,UrbanPercent,Employees-1131,Establishments-1131,...,Payroll-8132,Employees-8133,Establishments-8133,Payroll-8133,Employees-8134,Establishments-8134,Payroll-8134,Employees-8139,Establishments-8139,Payroll-8139
0,23001,Androscoggin County,107.32,-70.20,44.14,1287.54,0.08,0.0,NaN,NaN,...,NaN,46.0,6.0,1260.0,164.0,25.0,2562.0,17.0,10.0,946.0
1,23003,Aroostook County,68.84,-68.47,46.68,17683.29,0.00,0.0,NaN,NaN,...,229.0,26.0,3.0,970.0,26.0,10.0,330.0,74.0,11.0,1477.0
2,23005,Cumberland County,289.17,-77.28,40.15,3153.21,0.09,0.0,NaN,NaN,...,17203.0,465.0,66.0,25372.0,416.0,58.0,8695.0,457.0,77.0,18354.0
3,23007,Franklin County,30.18,-95.22,33.20,4516.56,0.01,0.0,NaN,NaN,...,NaN,49.0,6.0,1450.0,19.0,4.0,188.0,11.0,6.0,351.0
4,23009,Hancock County,54.47,-91.15,40.39,6073.34,0.01,0.0,NaN,NaN,...,3329.0,62.0,13.0,2916.0,58.0,10.0,1118.0,70.0,22.0,1685.0
5,23011,Kennebec County,121.29,-69.83,44.42,2463.53,0.05,0.0,NaN,NaN,...,5720.0,188.0,28.0,8171.0,125.0,26.0,2087.0,361.0,68.0,22986.0
6,23013,Knox County,39.70,-99.71,33.59,2962.80,0.01,0.0,NaN,NaN,...,474.0,103.0,16.0,4414.0,40.0,6.0,802.0,19.0,11.0,950.0
7,23015,Lincoln County,34.02,-81.21,35.49,1811.99,0.02,0.0,NaN,NaN,...,613.0,44.0,9.0,1865.0,82.0,5.0,1656.0,15.0,9.0,419.0
8,23017,Oxford County,57.23,-70.78,44.47,5635.11,0.01,0.0,NaN,NaN,...,174.0,2.0,5.0,188.0,20.0,6.0,306.0,79.0,11.0,1412.0
9,23019,Penobscot County,152.28,-68.60,45.29,9212.68,0.02,0.0,NaN,NaN,...,6003.0,75.0,17.0,2395.0,119.0,24.0,2628.0,51.0,18.0,1797.0


In [ ]:
"""
year_range=range(2017,2022)
state_list=['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
for k in [2,4]:
  for i in year_range:
    for j in state_list:
          try:
              main(i,j,k)
              print(f"naics{k}:{i}:{j} has been saved")
          except:
              continue
"""

'\nyear_range=range(2017,2022)\nstate_list=[\'AK\', \'AL\', \'AR\', \'AZ\', \'CA\', \'CO\', \'CT\', \'DC\', \'DE\', \'FL\', \'GA\',\n\'HI\', \'IA\', \'ID\', \'IL\', \'IN\', \'KS\', \'KY\', \'LA\', \'MA\', \'MD\', \'ME\',\n\'MI\', \'MN\', \'MO\', \'MS\', \'MT\', \'NC\', \'ND\', \'NE\', \'NH\', \'NJ\', \'NM\',\n\'NV\', \'NY\', \'OH\', \'OK\', \'OR\', \'PA\', \'RI\', \'SC\', \'SD\', \'TN\', \'TX\',\n\'UT\', \'VA\', \'VT\', \'WA\', \'WI\', \'WV\', \'WY\']\nfor k in [2,4]:\n  for i in year_range:\n    for j in state_list:\n          try:\n              main(i,j,k)\n              print(f"naics{k}:{i}:{j} has been saved")\n          except:\n              continue\n'